In [1]:
import pandas as pd

In [2]:
all_muts_df = pd.read_pickle("./data/2_8_df.pkl")
display(all_muts_df.shape, all_muts_df.head())

(2244, 39)

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,...,regulators,regulator links,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links
0,0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,...,"[{'name': 'PhoP'}, {'name': 'CRP'}, {'name': '...","{'CRP': ['ECK125162049'], 'NagC': ['ECK1251620...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR'}, {'name': 'NagC/TyrR'}]",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25'}, {'name': '3.5.1.25'}]","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}"
1,2,42C,1,124,1,1,1.0,1308318,SNP,G→C,...,[],{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{}
2,6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,...,[],{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO'}, {'name': 'purR-KO'}]","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10'}, {'name': '2.4.2.10'}]","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}"
3,7,42C,1,124,1,1,1.0,4187550,SNP,C→T,...,[],{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},[{'name': 'translation'}],{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'}, {'name': '2.7.7.6'}]","{'2.7.7.6': ['ECK120016992', 'ECK120016992']}"
4,8,42C,1,124,1,1,1.0,4400313,SNP,A→C,...,[{'name': 'CRP'}],{'CRP': ['ECK125147410']},[],{},[{'name': 'Signal transduction mechanisms'}],{'Signal transduction mechanisms': ['ECK125147...,[],{},[],{}


In [3]:
TU_objects_df = pd.read_pickle("./data/TU_objects_df.pkl")
gene_synonym_df = pd.read_pickle("./data/gene_synonym_df.pkl")
gene_pathway_df = pd.read_pickle("./data/gene_pathway_df.pkl")

In [4]:
gene_pathway_df.head()

,refseq_locus_tag,gene,product,ec_number,ec_description,pathway_name
0,b0002,thrA,Aspartokinase (EC 2.7.2.4) / Homoserine dehydr...,1.1.1.3,Homoserine dehydrogenase,"Glycine, serine and threonine metabolism"
1,b0002,thrA,Aspartokinase (EC 2.7.2.4) / Homoserine dehydr...,1.1.1.3,Homoserine dehydrogenase,Cysteine and methionine metabolism
2,b0002,thrA,Aspartokinase (EC 2.7.2.4) / Homoserine dehydr...,1.1.1.3,Homoserine dehydrogenase,Lysine biosynthesis
3,b0002,thrA,Aspartokinase (EC 2.7.2.4) / Homoserine dehydr...,2.7.2.4,Aspartate kinase,"Glycine, serine and threonine metabolism"
4,b0002,thrA,Aspartokinase (EC 2.7.2.4) / Homoserine dehydr...,2.7.2.4,Aspartate kinase,Cysteine and methionine metabolism


In [5]:
from util.gene import get_gene_bnum


def _get_products(gene_id):
    annots = []
    bnum = get_gene_bnum(gene_id, gene_synonym_df)
    df = gene_pathway_df[gene_pathway_df["refseq_locus_tag"]==bnum]
    for ec in df["product"]:
        d = {"name": ec}
        annots.append(d)
    return annots


def get_products(mut_row):
    annots = []
    for feat_d in mut_row["genomic features"]:
        # !!! This part only handles mutations that land within a genomic feature.
        # Need to also implement logic for handling mutations that land within intergenic regions.
        if feat_d["genetic"]:
            annots += _get_products(feat_d["RegulonDB ID"])
        # This will catch whether no features exist in the feat_ID_set
        # or if those features are just integenic regions.
        # Get operon genetic feature link dict through overlapping mutation and operon ranges.
        # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
        # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
        else:
            TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
            for TU_ID in TU_IDs:
                subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]
                for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                    if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                        annots += _get_products(gene_id)
    return annots

In [6]:
all_muts_df["products"] = all_muts_df.apply(get_products, axis=1)
all_muts_df.head()

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,...,regulator links,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links,products
0,0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,...,"{'CRP': ['ECK125162049'], 'NagC': ['ECK1251620...","[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR'}, {'name': 'NagC/TyrR'}]",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25'}, {'name': '3.5.1.25'}]","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}",[{'name': 'N-acetylglucosamine-6-phosphate dea...
1,2,42C,1,124,1,1,1.0,1308318,SNP,G→C,...,{},[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{},[]
2,6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,...,{},"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO'}, {'name': 'purR-KO'}]","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10'}, {'name': '2.4.2.10'}]","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}",[{'name': 'Orotate phosphoribosyltransferase (...
3,7,42C,1,124,1,1,1.0,4187550,SNP,C→T,...,{},"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},[{'name': 'translation'}],{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'}, {'name': '2.7.7.6'}]","{'2.7.7.6': ['ECK120016992', 'ECK120016992']}",[{'name': 'DNA-directed RNA polymerase beta' s...
4,8,42C,1,124,1,1,1.0,4400313,SNP,A→C,...,{'CRP': ['ECK125147410']},[],{},[{'name': 'Signal transduction mechanisms'}],{'Signal transduction mechanisms': ['ECK125147...,[],{},[],{},[]


In [7]:
TU_df = pd.read_pickle("./data/TU_df.pkl")
operon_df = pd.read_pickle("./data/operon_df.pkl")
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [8]:
from util.operon import get_operon_ID_set


def _add_links_to_d(bnum, d, op_ID_set):
    for ec in gene_pathway_df[gene_pathway_df["refseq_locus_tag"]==bnum]["product"]:
        if ec not in d.keys(): d[ec] = []
        d[ec] += list(op_ID_set)


def get_ec_operon_links_d(mut_row):
    ec_operon_d = dict()
    for feat_d in mut_row["genomic features"]:
        if feat_d["genetic"]:
            op_ID_set = get_operon_ID_set(feat_d["RegulonDB ID"], TU_objects_df, TU_df, operon_df)
            if str(feat_d["RegulonDB ID"]) != 'nan' and feat_d["RegulonDB ID"] != '':
                bnum = get_gene_bnum(feat_d["RegulonDB ID"], gene_synonym_df)
                _add_links_to_d(bnum, ec_operon_d, op_ID_set)
        else:
            TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
            for TU_ID in TU_IDs:
                subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]

                op_ID_set = set()
                for _, TU_gene in subset_TU_genes_df.iterrows():
                     op_ID_set = op_ID_set | get_operon_ID_set(TU_gene["TU_OBJECT_ID"], TU_objects_df, TU_df, operon_df)

                for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                    if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                        bnum = get_gene_bnum(gene_id, gene_synonym_df)
                        _add_links_to_d(bnum, ec_operon_d, op_ID_set)
    return ec_operon_d

                    
all_muts_df["product links"] = all_muts_df.apply(get_ec_operon_links_d, axis=1)
all_muts_df.head()

,index,exp,ale,flask,isolate,tech_rep,presence,Position,Mutation Type,Sequence Change,...,pathways,pathway links,COGs,COG links,imodulons,imodulon links,EC numbers,EC number links,products,product links
0,0,42C,1,124,1,1,1.0,702352,DEL,Δ21 bp,...,"[{'name': 'Galactose metabolism'}, {'name': 'A...","{'Galactose metabolism': ['ECK125162049'], 'Am...",[{'name': 'Carbohydrate transport and metaboli...,{'Carbohydrate transport and metabolism': ['EC...,"[{'name': 'NagC/TyrR'}, {'name': 'NagC/TyrR'}]",{'NagC/TyrR': ['nagBAC-umpH']},"[{'name': '3.5.1.25'}, {'name': '3.5.1.25'}]","{'3.5.1.25': ['ECK125162049', 'ECK125162049']}",[{'name': 'N-acetylglucosamine-6-phosphate dea...,{'N-acetylglucosamine-6-phosphate deacetylase ...
1,2,42C,1,124,1,1,1.0,1308318,SNP,G→C,...,[],{},[{'name': 'Lipid transport and metabolism'}],{'Lipid transport and metabolism': ['ECK125147...,[],{},[],{},[],{}
2,6,42C,1,124,1,1,1.0,3815859,DEL,Δ82 bp,...,"[{'name': 'Pyrimidine metabolism'}, {'name': '...","{'Pyrimidine metabolism': ['ECK120014627'], 'D...",[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...,"[{'name': 'purR-KO'}, {'name': 'purR-KO'}]","{'purR-KO': ['rph-pyrE', 'rph-pyrE']}","[{'name': '2.4.2.10'}, {'name': '2.4.2.10'}]","{'2.4.2.10': ['ECK120014627', 'ECK120014627']}",[{'name': 'Orotate phosphoribosyltransferase (...,{'Orotate phosphoribosyltransferase (EC 2.4.2....
3,7,42C,1,124,1,1,1.0,4187550,SNP,C→T,...,"[{'name': 'Purine metabolism'}, {'name': 'Pyri...","{'Purine metabolism': ['ECK120016992'], 'Pyrim...",[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']},[{'name': 'translation'}],{'translation': ['rplKAJL-rpoBC']},"[{'name': '2.7.7.6'}, {'name': '2.7.7.6'}]","{'2.7.7.6': ['ECK120016992', 'ECK120016992']}",[{'name': 'DNA-directed RNA polymerase beta' s...,{'DNA-directed RNA polymerase beta' subunit (E...
4,8,42C,1,124,1,1,1.0,4400313,SNP,A→C,...,[],{},[{'name': 'Signal transduction mechanisms'}],{'Signal transduction mechanisms': ['ECK125147...,[],{},[],{},[],{}


In [9]:
all_muts_df.to_pickle("./data/2_9_df.pkl")